## Import Pool-Model Package

In [ ]:
from cr_pool_model_pyo3 import *

import numpy as np

## Define Simulation Setings
These settings are predefined by our current simulation.
They directly control properties of the cells.
In this example, we focus on a limited subset of parameters which are relevant for our simulation.

In [ ]:
simulation_settings = SimulationSettings()

# Starting Domain
simulation_settings.starting_domain_x_low = 1400
simulation_settings.starting_domain_x_high = 1600
simulation_settings.starting_domain_y_low = 1400
simulation_settings.starting_domain_x_high = 1600

# Bacteria
simulation_settings.n_bacteria_initial = 30
# Bacteria Cycle
simulation_settings.bacteria_food_turnover_rate = 0.001
simulation_settings.bacteria_cycle_growth_rate = 0.01
simulation_settings.bacteria_cycle_food_growth_rate_multiplier = 0.5
# Bacteria Interactions & Mechanics
simulation_settings.bacteria_interaction_potential_strength = 10.0
simulation_settings.bacteria_mechanics_velocity_reduction = 0.25

# Domain
simulation_settings.voxel_food_initial_concentration = 30_000.0
simulation_settings.voxel_food_diffusion_constant = 10.0

# General Settings
simulation_settings.dt = 0.005
simulation_settings.n_times = 25_001
simulation_settings.n_threads = 8
print(simulation_settings)

In [ ]:
output_path = run_simulation(simulation_settings)
# output_path = "out/pool_model/2023-10-31-20:06:34"

## Read results from json Files
The results of the simulation are saved in json files.
Due to the parallelized nature of the simulation, not all results are in one big json file but rather in multiple batches. We therefore need to combine these batches to obtain a complete set for a given iteration.

In [ ]:
import os
from pathlib import Path
import json

def combine_batches(run_directory):
    # Opens all batches in a given directory and stores
    # them in one unified big list
    combined_batch = []
    for batch_file in os.listdir(run_directory):
        f = open(run_directory / batch_file)
        b = json.load(f)["data"]
        combined_batch.extend(b)
    return combined_batch

def get_cells_at_iterations(output_path):
    # Uses the previously defined funtion [combine_batches]
    # to read all stored cells at all iterations and stores
    # them in a dictionary.
    dir = Path(output_path) / "cell_storage/json/"
    runs = [(x, dir / x) for x in os.listdir(dir)]
    result = []
    for (n_run, run_directory) in runs:
        result.extend([{"iteration":int(n_run)} | c for c in combine_batches(run_directory)])
    return result

cells_at_iter = get_cells_at_iterations(output_path)

We want to inspect which entries our generated dataset has. Therefore, we normalize the dict, transforming it into a dataframe.
Afterwards, we display all columns.

In [ ]:
import pandas as pd

df = pd.json_normalize(cells_at_iter)
for col in df.columns:
    print(col)

In [ ]:
data = np.array([[itera, len(group)] for (itera, group) in df.groupby("iteration")])

## Plot Colony
We plot the bacterial colony using `matplotlib`. All particles are represented by 2D spheres and so we can display them as such.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import multiprocessing as mp

In [ ]:
fig, ax = plt.subplots()

ax.plot(data[:,0], data[:,1], label="Bacteria Count")
ax.legend()
plt.show(fig)

In [ ]:
def save_snapshot(iteration):
    # Filter for only particles at the specified iteration
    df_filtered = df[df["iteration"]==iteration]

    # Get positions as large numpy array
    positions = np.array([np.array(x) for x in df_filtered["element.cell.mechanics.pos"]])
    s = np.array([np.array(x) for x in df_filtered["element.cell.interaction.cell_radius"]])
    c = np.array([np.array(x) for x in df_filtered["element.cell.cellular_reactions.intracellular_concentrations"]])
    norm = matplotlib.colors.Normalize(
        vmin=c.min(),
        vmax=c.max(),
        clip=True,
    )
    mapper = matplotlib.cm.ScalarMappable(norm=norm, cmap=matplotlib.cm.summer)
    c = mapper.to_rgba(c)

    x_low = positions[:,0].min()
    x_high = positions[:,0].max()
    x_middle = (x_low+x_high)/2
    y_low = positions[:,1].min()
    y_high = positions[:,1].max()
    y_middle = (y_low+y_high)/2.0

    factor = 1.1
    xlims=[x_middle + factor*(x_low-x_middle), x_middle + factor*(x_high-x_middle)]
    ylims=[y_middle + factor*(y_low-y_middle), y_middle + factor*(y_high-y_middle)]

    ratio = (xlims[1]-xlims[0])/(ylims[1]-ylims[0])
    figsize = (8, 8*ratio)
    fig, ax = plt.subplots(figsize=figsize)

    for pos, si, ci in zip(positions, s, c):
        circle = plt.Circle(pos, radius=si, facecolor=ci, edgecolor='k')
        ax.add_patch(circle)

    ax.set_xlim(xlims)
    ax.set_ylim(ylims)

    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    fig.tight_layout()

    fig.savefig(Path(output_path) / "snapshot_{:08}.png".format(iteration))
    plt.close(fig)

with mp.Pool() as p:
    p.map(save_snapshot, np.unique(df["iteration"]))